In [2]:
import pandas as pd
import googlemaps
import os
from dotenv import load_dotenv, find_dotenv


In [3]:
# find .env automagically by walking up directories until it's found
dotenv_path = find_dotenv()
# load up the entries as environment variables
load_dotenv(dotenv_path)
# get the path of data
path_interim_data = os.environ.get("PATH_INTERIM_DATA_CLEANED")
path_processed_data = os.environ.get("PATH_PROCESSED_DATA")
# get google maps key
gmaps_key = os.environ.get("GOOGLE_MAPS_KEY")

# set filename
filename = 'crimes.csv'

# set country 
country = 'BR'

# set city name
city = 'São Carlos'


In [4]:
# load data
path_data = path_interim_data + filename
data = pd.read_csv(path_data,index_col='index')
noref = data['LATITUDE'] == 0
data_noref = data[noref]
data_noref['Num_logradouro'] = data_noref['Num_logradouro'].astype(int).astype(str)
data_noref['CEP'] = data_noref['CEP'].astype(str) 
data_noref.fillna('',inplace=True)
data_noref.replace('nan', '', inplace=True)
data_noref.head()

C:\Users\Tiago\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Tiago\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\Tiago\Anaconda3\lib\site-packages\pandas\core\frame.py:4244: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retur

,Ano,Num_BO,Data_ocorrencia_BO,Hora_ocorrencia_BO,Data_hora_registro_BO,Data_hora_impressao_BO,Período,Flagrante,Status,Conduta,Bairro,Tipo_local,CEP,LOGRADOURO,Num_logradouro,LATITUDE,LONGITUDE
index,,,,,,,,,,,,,,,,,
11,2017,644,2017-09-21,18:00:00,2017-09-23 13:27:17.000000,2017-09-23 13:54:17.000000,,N,Consumado,TRANSEUNTE,,RESTAURANTE E AFINS,,AVENIDA GETULIO VARGAS,1,0.0,0.0
16,2019,596,2019-02-05,10:15:00,2019-02-06 01:18:47.000000,2019-02-06 03:11:05.000000,,N,Consumado,VEICULO,RURAL,RODOVIA/ESTRADA,,RODOVIA SP 310,265,0.0,0.0
23,2017,9,2017-01-18,,2017-01-18 16:08:03.000000,2017-01-20 17:09:43.000000,DE MADRUGADA,N,Consumado,VEICULO,,RODOVIA/ESTRADA,,RODOVIA WASHINGTON LUIZ KM 221,221,0.0,0.0
24,2015,1276,2015-12-20,,2015-12-21 12:15:28.000000,2015-12-21 12:37:30.000000,EM HORA INCERTA,N,Consumado,VEICULO,,VIA PÚBLICA,,RUA CLAUDENICE BALDAN MAIELO,227,0.0,0.0
34,2014,970,2014-08-11,18:00:00,2014-08-12 11:38:57.000000,2014-08-12 14:36:44.000000,,N,Consumado,VEICULO,DISTRITO INDUSTRIAL,VIA PÚBLICA,14820000,AVENIDA HERMINIO CRISTOVÃO,395,0.0,0.0


In [5]:
gmaps = googlemaps.Client(key=gmaps_key, queries_per_second=40)

components = {
    'country': country,
    'administrative_area': city
}

In [6]:
no_results = []
more_than_one = []
partial_match = []

for index, row in data_noref.iterrows():
    try:
            
            address = row.LOGRADOURO + ',' + row.Num_logradouro + ',' + city
            print(address)
            result = gmaps.geocode(language='pt-BR', address= address, components=components)
            if not result:
                    print('No results')
                    no_results.append(row)
                    continue
            elif len(result) > 1:
                    print('More than one')
                    more_than_one.append(row)
                    continue
            elif result[0].get('partial_match') != None:
                    print('Partial Match')
                    partial_match.append(row)
                    continue
            else:
                data_noref.loc[index, 'LATITUDE'] = result[0]['geometry']['location']['lat']
                data_noref.loc[index, 'LONGITUDE'] = result[0]['geometry']['location']['lng']
    except Exception as e:
        print(e)

AVENIDA GETULIO VARGAS,1,São Carlos


C:\Users\Tiago\Anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


RODOVIA SP 310,265,São Carlos
RODOVIA WASHINGTON LUIZ  KM 221,221,São Carlos
More than one
RUA CLAUDENICE BALDAN MAIELO,227,São Carlos
Partial Match
AVENIDA HERMINIO CRISTOVÃO,395,São Carlos
Partial Match
AVENIDA JOAQUIM PINTO PEREIRA DE ALMEIDA,585,São Carlos
RUA PERIMETRAL PROJETADA,81,São Carlos
More than one
RUA ALEXANDRE PEDRAZZANI,190,São Carlos
RUA PADRE TEIXEIRA - CLUBE FLOR DE MAIO,1,São Carlos
RUA SECOMBINA DE PAULA PASSADOR,393,São Carlos
RUA ANTONIO SELARIM,80,São Carlos
RUA NUNCIO CARDINALI,4,São Carlos
RUA GERMANO FEHR JUNIOR,435,São Carlos
R CAP LUIZ BRANDAO,291,São Carlos
RUA RUI BARBOSA,102,São Carlos
RUA MAJOR MANOEL ANTONIO DE MATTOS,195,São Carlos
RUA JOSÉ LEMES MARQUES,170,São Carlos
RUA PADRE TEIXEIRA,2431,São Carlos
RUA FAGUNDES VARELA,305,São Carlos
RUA SETE DE SETEMBRO,3040,São Carlos
RUA ANTONIO GIGLIOTI,226,São Carlos
Partial Match
RUA TOTÓ LEITE,30,São Carlos
RUA RAIMUNDO CORREA,845,São Carlos
RUA ANTONIO BLANCO,297,São Carlos
AVENIDA GETULIO VARGAS,1875,São

RUA  JOSE ANTONIO MIGLIATTO,654,São Carlos
RUA N,1580,São Carlos
Partial Match
VICINAL WASHINGTON JOSE PERA,25,São Carlos
Partial Match
AVENIDA PERIMETRAL,1425,São Carlos
RUA CESARE BRIGANTE,378,São Carlos
RUA ANTONIO LUIZ ZANQUIM,438,São Carlos
More than one
RUA GEOVANE VASSOLO,120,São Carlos
RUA ORESTES MASTRO FRANCISCO,135,São Carlos
RUA ANTONIO CARLOS DELA DEA,440,São Carlos
RUA FRANCISCO FERNARDES,39,São Carlos
RUA REGIT ARAB,470,São Carlos
AVENIDA REGIT ARAB,1185,São Carlos
RUA VANDERLEI QUELE DE LIMA,240,São Carlos
RUA DESESEIS,95,São Carlos
Partial Match
ESTRADA MUNICIPAL DOMINGOS GIANOTI,440,São Carlos
Partial Match
RUA 18,89,São Carlos
RUA CASEMIRO DE ABREU,196,São Carlos
RUA JOAQUIM GARCIA DE OLIVEIRA,849,São Carlos
RUA VINTE E UM,268,São Carlos
RUA JOAO LUCAS,50,São Carlos
Partial Match
RUA JOAQUIM GARCIA DE OLIVEIRA,260,São Carlos
RUA AMIR SOARES QUATROCH,96,São Carlos
RUA RUBENS FERNANDO MONTE RIBEIRO,485,São Carlos
RUA NILO SERGIO PAULINO,150,São Carlos
More than one
RUA

RUA DOUTOR ORLANDO DAMIANO,2411,São Carlos
RUA SETE DE SETEMBRO,1787,São Carlos
AVENIDA CLEMENTE TALARICO,600,São Carlos
AVENIDA QUINZE DE NOVEMBRO,99999,São Carlos
PASSEIO  DOS IPES,1,São Carlos
PASSEIO  DOS IPÊS,1,São Carlos
RUA ORLANDO DE OLIVEIRA,735,São Carlos
RODOVIA SP 215,166,São Carlos
More than one
RODOVIA SP 215,166,São Carlos
More than one
RUA 13 DE MAIO,1930,São Carlos
AVENIDA SALLUM,1201,São Carlos
RUA MANOEL FRAGOAS,318,São Carlos
AV DR TEIXEIRA DE BARROS,1026,São Carlos
RUA SETE DE SETEMBRO,2948,São Carlos
RUA JAIME BRUNO,127,São Carlos
RUA RAY WESLLEY HERICK,759,São Carlos
Partial Match
RUA HEUVÍDEO GOUVEIA,396,São Carlos
Partial Match
RUA SETE DE SETEMBRO,2763,São Carlos
RUA PEDRO RAIMUNDO,95,São Carlos
RUA DOMINGOS JORGE VELHO,70,São Carlos
RUA DOM CARMINI ROCCO,55,São Carlos
RUA JESUINO DE ARRUDA,1649,São Carlos
RUA AMERICO GIACOMINO CANHOTO,175,São Carlos
Partial Match
RUA DESEMB. JULIO DE FARIA,948,São Carlos
AV REPUBLICA DO LÍBANO,340,São Carlos
RUA MARECHAL DEOD

LOTEAMENTO NOVA SÃO CARLOS,1,São Carlos
RUA PASSEIO DOS FLABOYANTS,200,São Carlos
RUA MAJOR JÚLIO SALES,492,São Carlos
RUA FERNAO DIAS,190,São Carlos
AVENIDA PASSEIO DOS FLAMBOIANTS,200,São Carlos
AVENIDA  BRUNO RUGGIERO FILHO,1751,São Carlos
PASSEIO  DOS IPES,140,São Carlos
RUA ALDERICO VIEIRA PERDIGAO,227,São Carlos
RUA 7 DE SETEMBRO,3033,São Carlos
RUA MARIA CONSUELO BRANDAO TOLENTINO,424,São Carlos
AVENIDA GERMANO FHER JUNIOR,1,São Carlos
AVENIDA DON CARMINE ROCO,455,São Carlos
PASSEIO  DOS FLAMBOYANTS,200,São Carlos
AVENIDA BRUNO RUGIERO FILHO,821,São Carlos
RUA PASSEIO DOS FLABOYANT,200,São Carlos
RUA PASSEIO DOS FLABOYANT,200,São Carlos
RODOVIA SP310,235,São Carlos
Partial Match
RUA JOSE ZAGATO,90,São Carlos
RUA HILARIO MARTINS DIAS,61,São Carlos
RUA HILARIO MARTINS DIAS,61,São Carlos
RUA ESPICOPAL,965,São Carlos
AVENIDA DAS AZALEIAS,140,São Carlos
RUA JOSE MANCINI,616,São Carlos
TERMINAL PONTO FINAL DEF. RESIDENCIAL TE,76,São Carlos
Partial Match
RUA THEODOLINA MODENA COCA,218,

Avenida Comendador Alfredo Maffei,700,São Carlos
Rua Antonio Botelho,561,São Carlos
Rua das Orquideas,564,São Carlos
RUA DA IMPRENSA,206,São Carlos
rua episcopal,994,São Carlos
Rua Antonio de Almeida Leite,930,São Carlos
AV.CDOR OSCAR FERREIRA,2030,2030,São Carlos
rua 7 de setembro,2000,São Carlos
Avenida Clemente Talarico,500,São Carlos
r. dr carlos botelho,2474,São Carlos
Partial Match
RUA ELÍAS ARSENIOS,1178,São Carlos
RUA PEDRO JOSÉ NETO,219,São Carlos
AVENIDA SÃO CARLOS,3800,São Carlos
RUA SANTA GERTRÚDES,1052,São Carlos
RUA CID SILVA CÉSAR,126,São Carlos
RUA CÉSAR RICOMI,3055,São Carlos
Partial Match
RUA DOUTOR CARLOS DE CAMARGO SALES,222,São Carlos
RUA ANTONIO PRATA VIEIRA,50,São Carlos
RUA SÃO JOAQUIM,13575,São Carlos
AVENIDA SALUM,249,São Carlos
bernadino de campos,2000,São Carlos
rua ana prado,529,São Carlos
Rua Germiniano Costa x Av: São Carlos,650,São Carlos
Partial Match
Rua Manoel Peres Dias,192,São Carlos
padre teixera,2363,São Carlos
Rua Dr. Gastao de Sa,286,São Carlos


In [10]:
#save file
data_noref = data_noref[data_noref.LATITUDE != 0]
data_noref.to_csv(path_processed_data+filename, index=False)
#data.update(data_noref)
#data.to_csv(path_processed_data+filename, index=False)

In [20]:
import folium
crimes = data_noref
def in_San_Charles(lat, lng):
    return lat > -21 and lat < -23 and lng < -48 and lng > -47


map = folium.Map(location=[-22.010147, -47.890706], zoom_start=13)
latitudes = list(crimes['LATITUDE'])
longitudes = list(crimes['LONGITUDE'])
logradouros = list(crimes['LOGRADOURO'])
n_logradouros = list(crimes['Num_logradouro'])
out_of_bounds = 0

for lat, lng, logradouro, num in zip(latitudes, longitudes, logradouros, n_logradouros):
    if not in_San_Charles(lat, lng):
        out_of_bounds += 1
        folium.Marker(location=[lat, lng], tooltip=logradouro+'-'+num, popup=[lat, lng]).add_to(map)
        
print(out_of_bounds)

765


In [21]:
map